# Bokeh Exercises

*The "User Guide" linked on this page is a very good resource :* https://bokeh.pydata.org/en/latest/


## 1. Create a single Bokeh plot of some data 

*Include the standard pan, wheel_zoom, box_zoom, reset, save, box_select, and lasso_select tools, but no other special widgets*

## 2. Create a single Bokeh plot of some data with some widget

*Include the standard pan, wheel_zoom, box_zoom, reset, save, box_select, and lasso_select tools, and also a widget (e.g., slider, dropdown selection, etc.)  that controls some aspect of the plot*

In [151]:
### Todo ###
############
    # Drop-down for selecting anion
    # Different colors for predicted d states? rows on ptable?
    # Drop-down for plotting different axes?

import numpy as np
import pandas as pd
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CustomJS, Slider, Button, DataTable, TableColumn, NumberFormatter
from bokeh import events
from bokeh.io import curdoc
from bokeh.models.tools import HoverTool

output_notebook()
output_file("tmc.html", title='Transition Metal Compounds')

Loading BokehJS ...

In [152]:
### Import, prepare data
df = pd.read_csv('../datasets/features_icsd_tmetal-compounds.csv')
df = df[df.heteroanion == False]
df = df[df.anions != np.nan]
source = ColumnDataSource(data=dict(x0=df.mm_dist, x1=df.normed_dist, y=df.gii, formula=df.formula, 
                                    sg_sym=df.sg_sym, pred_d1=df.pred_d1, delta=df.delta, n1=df.n1))
# df.keys()

In [154]:
### Create plot
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

### Metal-Metal Distance vs GII
left = figure(tools=TOOLS, width=450, height=450, x_range=(0, 9), y_range=(-0.01, 1), 
           title="Metal-Metal Distance vs. GII")
left.title.align = 'center'
left.circle('x0', 'y', source=source, color='black', fill_alpha=0, line_alpha=0.2, 
         legend_label='ICSD TMCs')
left.xaxis.axis_label = "Metal-Metal Distance (Angstroms)"
left.yaxis.axis_label = "Global Instability Index"
left.xaxis.axis_label_text_font_style = "normal"
left.yaxis.axis_label_text_font_style = "normal"
# Shaded area
left.patch([-9999,-9999,9999,9999],[-0.01,0.2,0.2,-0.01],alpha=0.2, line_width=0, 
        legend_label='Current Stability Metric')
# Legend
left.legend.background_fill_alpha = 0.5

### Delta vs GII
right = figure(tools=TOOLS, width=400, height=450, x_range=(0.4, 3), y_range=left.y_range, 
           title="Normalized Metal-Metal Distance vs. GII")
right.title.align = 'center'
right.circle('x1', 'y', source=source, color='black', fill_alpha=0, line_alpha=0.2)
right.xaxis.axis_label = "Metal-Metal Distance / Pure Metal "
right.yaxis.visible = False
right.xaxis.axis_label_text_font_style = "normal"
# Shaded area
right.patch([-9999,-9999,9999,9999],[-0.01,0.2,0.2,-0.01],alpha=0.2, line_width=0)

### Tooltips
hover = HoverTool()
hover.tooltips ="""
    <div>
        <h3>@formula</h3>
        <div><strong>Spacegroup:  </strong>@sg_sym</div>
        <div><strong>Predicted d state:  </strong>@pred_d1</div>
    </div>
"""
left.add_tools(hover)
right.add_tools(hover)

p = gridplot([[left, right]])

### Define the layout and show the interactive
# widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
show(p)